In [1]:
# First, add your HuggingFace Token (with Write access) to Google Colab Secrets
# Then, load the token using below code

from huggingface_hub import login
from google.colab import userdata

hf_token = userdata.get('HF_API_KEY')
login(token = hf_token)

In [2]:
#@title Download Dataset
from IPython.display import clear_output
!gdown https://drive.google.com/uc?id=1Q1r1FGDaBQLkvPKMjvgnA2eWBpmMLD57
clear_output()
!ls | grep '.csv'

clean_MedQuAD_autotrain_ready.csv


In [3]:
import pandas as pd

df = pd.read_csv("clean_MedQuAD_autotrain_ready.csv")
df.shape
df.head()

,Question,Answer
0,what is (are) age-related macular degeneration ?,"age-related macular degeneration, also known a..."
1,what are the treatments for age-related macula...,"wet amd can be treated with laser surgery, pho..."
2,what is (are) age-related macular degeneration ?,the daily amounts used by the study researcher...
3,who is at risk for age-related macular degener...,risk increases with age amd is most common in ...
4,what is (are) age-related macular degeneration ?,drusen are yellow deposits under the retina. t...


In [4]:
# Load your fine-tuned model

from sentence_transformers import SentenceTransformer

# Load embedding model
fine_tuned_model = SentenceTransformer("rajucanon/mini-project-medical-assistant2")      # <------ change this as per your model id
                                                                                # This finetuned model is standing on
                                                                                # Base Model: sentence-transformers/all-MiniLM-L6-v2


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/199 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [5]:
# Load your fine-tuned model

from sentence_transformers import SentenceTransformer

# Load embedding model
untuned_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
def compute_embeddings(x, model):
    return model.encode(x)


In [7]:
# Create Fine-tuned embeddings & Untuned embeddings for the `Question`

df["fine_tuned_embeddings"] = df["Question"].apply(lambda x: compute_embeddings(x, fine_tuned_model))
df["untuned_embeddings"] = df["Question"].apply(lambda x: compute_embeddings(x,untuned_model))

print("Embeddings ready for", len(df), "questions.")

Embeddings ready for 520 questions.


In [8]:
df.head()

,Question,Answer,fine_tuned_embeddings,untuned_embeddings
0,what is (are) age-related macular degeneration ?,"age-related macular degeneration, also known a...","[-0.026190035, -0.0017176677, 0.056867566, -0....","[0.0020649016, 0.013483159, 0.03716581, -0.006..."
1,what are the treatments for age-related macula...,"wet amd can be treated with laser surgery, pho...","[-0.0055234586, -0.0056177387, 0.09334571, -0....","[0.016901245, 0.0075773853, 0.062568605, -0.01..."
2,what is (are) age-related macular degeneration ?,the daily amounts used by the study researcher...,"[-0.026190035, -0.0017176677, 0.056867566, -0....","[0.0020649016, 0.013483159, 0.03716581, -0.006..."
3,who is at risk for age-related macular degener...,risk increases with age amd is most common in ...,"[0.022231508, -0.009564441, 0.05469262, -0.031...","[0.03975155, 0.004256208, 0.04601966, -0.01973..."
4,what is (are) age-related macular degeneration ?,drusen are yellow deposits under the retina. t...,"[-0.026190035, -0.0017176677, 0.056867566, -0....","[0.0020649016, 0.013483159, 0.03716581, -0.006..."


In [18]:
import numpy as np

def get_answer(user_question, model, top_k=1):
    # Encode user question
    q_emb = model.encode(user_question)

    if model == fine_tuned_model:
        # Convert embeddings column to numpy array
        all_embs = np.vstack(df["fine_tuned_embeddings"].values)

    else:
        # Convert embeddings column to numpy array
        all_embs = np.vstack(df["untuned_embeddings"].values)


    # Compute cosine similarities (vectorized for speed)
    sims = np.dot(all_embs, q_emb) / (
        np.linalg.norm(all_embs, axis=1) * np.linalg.norm(q_emb)
    )
    #----------------------------------
    # Get indices of top-k matches
    top_idx = np.argsort(sims)[::-1][:top_k]

    # Build response
    results = []
    for idx in top_idx:
        results.append(
            f"**Similarity: {sims[idx]:.2f}** → Answer: {df.iloc[idx]['Answer']}"
        )

# Return matched top 3 answers
    return "\n\n".join(results)


In [14]:
# Test the fine-tuned model

get_answer("Can lifestyle changes reduce the risk of developing type 2 diabetes?", model = fine_tuned_model)

In [11]:
# Test the untuned model

get_answer("Can lifestyle changes reduce the risk of developing type 2 diabetes?", model = untuned_model)

'**Similarity: 0.60** → Answer: diabetes is a serious, life-long disease. it can lead to problems such as heart disease, stroke, vision loss, kidney disease, and nerve damage. more than 8 million people in the united states have type 2 diabetes and dont know it. many people dont find out they have diabetes until they are faced with problems such as blurry vision or heart trouble. certain factors can increase your risk for diabetes, and its important to know what they are. type 1 diabetes type 1 diabetes is an autoimmune disease. in an autoimmune reaction, antibodies, or immune cells, attach to the bodys own healthy tissues by mistake, signaling the body to attack them. at present, scientists do not know exactly what causes the body\'s immune system to attack the cells, but many believe that both genetic factors and environmental factors, such as viruses, are involved. studies are now underway to identify these factors and prevent type 1 diabetes in people at risk. learn more about the 

In [15]:
import gradio as gr

# Function to get output from Fine-tuned model
def get_response(input):
    response = get_answer(input, model = fine_tuned_model)
    return response


# Input element
in_ques = gr.Textbox(label="Ask a medical question", placeholder="Type your question here...", lines=5)

# Output element
out_ans = gr.Textbox(label="Answer", lines = 10)


# Create Gradio Interface
iface = gr.Interface(fn = get_response,
                     inputs = [in_ques],
                     outputs = [out_ans],
                     title = "Medical Q&A",
                     description = "Application to answer medical questions using a fine-tuned model",
                     flagging_mode = "never")
# Launch interface
iface.launch()          # set `debug=True` while debugging

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7cacc753991244715c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
